In [119]:
%run -i opt_model.py

In [73]:
m.run_till(100)

returned t_delta = -0.70347493783175 < 0, set to 0 instead.
returned t_delta = -0.12213105245918321 < 0, set to 0 instead.
returned t_delta = -0.33803529002725297 < 0, set to 0 instead.
returned t_delta = -0.03607620241669007 < 0, set to 0 instead.
returned t_delta = -0.19146880277971334 < 0, set to 0 instead.
returned t_delta = -0.43657462238437894 < 0, set to 0 instead.
returned t_delta = -0.08819832360195612 < 0, set to 0 instead.
returned t_delta = -0.2891079019521632 < 0, set to 0 instead.
returned t_delta = -1.4936580139576723 < 0, set to 0 instead.
returned t_delta = -0.008027176538622172 < 0, set to 0 instead.
returned t_delta = -0.3349115535350773 < 0, set to 0 instead.
returned t_delta = -1.3895493537704748 < 0, set to 0 instead.
returned t_delta = -0.5578138185681976 < 0, set to 0 instead.
returned t_delta = -1.555400754326051 < 0, set to 0 instead.
returned t_delta = -0.0453411046956127 < 0, set to 0 instead.
returned t_delta = -0.01442285597097026 < 0, set to 0 instead.
re

In [74]:
sum(1 for ev in m.state.sinks[1000] if ev.src_id == 1)

116

In [75]:
sum(1 for ev in m.state.sinks[1000] if ev.src_id == 2)

84

In [76]:
sum(1 for ev in m.state.sinks[1000] if ev.src_id == 3)

128

In [77]:
np.random.rand()

0.24773098950115746

In [120]:
m2 = Manager([1000], [Poisson(1), Hawkes(2), Opt(3)])

In [121]:
m2.run_till(100);

returned t_delta = -0.70347493783175 < 0, set to 0 instead.
returned t_delta = -0.12213105245918321 < 0, set to 0 instead.
returned t_delta = -0.33803529002725297 < 0, set to 0 instead.
returned t_delta = -0.03607620241669007 < 0, set to 0 instead.
returned t_delta = -0.40788230608510023 < 0, set to 0 instead.
returned t_delta = -1.279237283061815 < 0, set to 0 instead.
returned t_delta = -0.24704811780943103 < 0, set to 0 instead.
returned t_delta = -0.2194684516867884 < 0, set to 0 instead.
returned t_delta = -1.3802919353697352 < 0, set to 0 instead.
returned t_delta = -0.5659569719177977 < 0, set to 0 instead.
returned t_delta = -1.5472576009764492 < 0, set to 0 instead.
returned t_delta = -0.053445729107927775 < 0, set to 0 instead.
returned t_delta = -0.006318231558655185 < 0, set to 0 instead.
returned t_delta = -0.13546239915728364 < 0, set to 0 instead.
returned t_delta = -0.070896020933219 < 0, set to 0 instead.
returned t_delta = -0.10476577845499335 < 0, set to 0 instead.
r

In [122]:
df = m2.state.get_dataframe()

In [125]:
df['t'] = np.cumsum(df['time_delta'])

In [126]:
df.head()

,event_id,sink_id,src_id,time_delta,t
0,100,1000,3,0.000000,0.000000
1,101,1000,1,0.469268,0.469268
2,102,1000,3,0.443674,0.912943
3,103,1000,1,0.873071,1.786014
4,104,1000,3,0.000000,1.786014
